# EDA
В датасете собраны данные с двух отелей: курортного и городского

Данные содержат *«бронирования, которые должны быть получены в период с 1 июля 2015 года по 31 августа 2017 года».*
**Примечание**. Для большинства вопросов я буду использовать только те бронирования, которые не были отменены, чтобы получить актуальные номера гостей.  Как вы увидите, это довольно большая разница.

Вопросы на которые нужно ответить исходя из данных:
- Откуда приходят гости?
- Сколько гости платят за номер в сутки?
- Как меняется цена за ночь в течение года?
- Какой самый загруженный месяц?
- Как долго люди остаются в отелях?
- Сколько бронирований было отменено?
- В каком месяце больше всего бронирований?


# Imports

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [42]:
df = pd.read_csv('./dataset/hotel_bookings.csv')
df.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,No Deposit,NaN,NaN,0,Transient,107.0,0,0,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,...,No Deposit,303.0,NaN,0,Transient,103.0,0,1,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,...,No Deposit,240.0,NaN,0,Transient,82.0,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,...,No Deposit,15.0,NaN,0,Transient,105.5,0,0,Canceled,2015-04-22


In [43]:
df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [44]:
df.shape

(119390, 32)

# Откуда приходят гости?

In [45]:
data = pd.DataFrame(df.loc[df["is_canceled"] == 0]["country"].value_counts())
data['names'] = data.index
total = data['country'].sum()
data['values'] = round(data['country'] / total * 100, 2)

plot = px.pie(data,
    title = 'Страны посетителей отелей',
    values = 'values',
    names = 'names',
    template="seaborn"
)

plot.update_traces(textposition="inside", textinfo="value+percent+label")
plot.show()

In [46]:
guest_map = px.choropleth(data,
                    locations=data['names'],
                    color=data["values"],
                    hover_name=data['names'],
                    color_continuous_scale='magma',
                    title="Страны посетителей отелей")
guest_map.show()

# Сколько гости платят за номер в сутки?
Предположим что цены указаны в Еврах. В отелях платными постояльцаи считаются дети и взрослые, младенцы бесплатные

In [67]:
rh = df.loc[(df["hotel"] == "Resort Hotel") & (df["is_canceled"] == 0)]
ch = df.loc[(df["hotel"] == "City Hotel") & (df["is_canceled"] == 0)]

In [68]:
rh["adr_not_free"] = rh["adr"] / (rh["adults"] + rh["children"])
ch["adr_not_free"] = ch["adr"] / (ch["adults"] + ch["children"])

/tmp/ipykernel_22739/2442919705.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_22739/2442919705.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [69]:
df["adr_not_free"] = df["adr"] / (df["adults"] + df["children"])
df = df.loc[df["is_canceled"] == 0]

In [71]:
print(f'Средняя стоимость одной ночи в отеле составляет:\n Курортный отель: {rh.adr_not_free.mean()}\n Городской отель: {ch.adr_not_free.mean()}')

Средняя стоимость одной ночи в отеле составляет:
 Курортный отель: inf
 Городской отель: inf
